# Imports

In [1]:
import csv  # for slang
import os
import re  # regex
import string  # punct
from pprint import pprint
!pip install emoji --upgrade
import emoji  # for emoji
import gensim
import keras
import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from gensim.models import Word2Vec
from IPython.display import Image
from matplotlib import pyplot as plt
from nltk.corpus import stopwords  # stopwords
from nltk.stem import PorterStemmer  # stemming
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn import svm, tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             brier_score_loss, classification_report,
                             confusion_matrix, f1_score, fbeta_score,
                             make_scorer,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler


# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#from google.colab import files
#uploaded = files.upload()
import nltk
nltk.download('stopwords')

%matplotlib inline

Requirement already up-to-date: emoji in c:\users\eleftheria\appdata\local\continuum\anaconda3\lib\site-packages (0.5.4)


Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eleftheria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Homemade Classes and Functions


In [2]:
# Clean Text Class

class CleanText(BaseEstimator, TransformerMixin):
    
    def remove_mentions(self, input_text):
        '''
        Remove mentions, like @Mplamplampla
        '''
        return re.sub(r'@+', '', input_text)
    
    def remove_urls(self, input_text):
        '''
        Remove the urls mention in a tweet
        '''
        input_text  = ' '.join([w for w in input_text.split(' ') if '.com' not in w])
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        input_text = emoji.demojize(input_text)
        input_text = input_text.replace('_','')
        input_text = input_text.replace(':','')
        return input_text
    
    def possessive_pronouns(self, input_text):
        '''
        Remove the possesive pronouns, because otherwise after tokenization we will end up with a word and an s
        Example: government's --> ["government", "s"]
        '''
        return input_text.replace("'s", "")
    
    def characters(self, input_text):
        '''
        Remove special and redundant characters that may appear on a tweet and that don't really help in our analysis
        '''
        input_text = input_text.replace("\r", " ") # Carriage Return
        input_text = input_text.replace("\n", " ") # Newline
        input_text = " ".join(input_text.split()) # Double space
        input_text = input_text.replace('"', '') # Quotes
        return input_text
    
    def remove_punctuation(self, input_text):
        '''
        Remove punctuation and specifically these symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        '''
        punct = string.punctuation # string with all the punctuation symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        '''
        Remove numbers
        '''
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        '''
        Convert all the sentences(words) to lowercase
        '''
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        '''
        Remove stopwords (refers to the most common words in a language)
        '''
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        '''
        Reduce the words to their stem
        '''
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def encode_decode(self, input_text):
        '''
        Remove weird characters that are result of encoding problems
        '''
        return  " ".join([k.encode("ascii", "ignore").decode() for k in input_text.split(" ")])
    
    
    def translator(self, input_text):
        '''
        Transform abbrevations to normal words
        Example: asap --> as soon as possible
        '''
        input_text = input_text.split(" ")
        j = 0
        for _str in input_text:
            # File path which consists of Abbreviations.
            fileName = r"C:\Users\Eleftheria\Desktop\slang.txt"
            # File Access mode [Read Mode]
            accessMode = "r"
            with open(fileName, accessMode) as myCSVfile:
                # Reading file as CSV with delimiter as "=", so that 
                # abbreviation are stored in row[0] and phrases in row[1]
                dataFromFile = csv.reader(myCSVfile, delimiter="=")
                # Removing Special Characters.
                _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
                for row in dataFromFile:
                    # Check if selected word matches short forms[LHS] in text file.
                    if _str.upper() == row[0]:
                        # If match found replace it with its appropriate phrase in text file.
                        input_text[j] = row[1]
                myCSVfile.close()
            j = j + 1
        
        return(' '.join(input_text))
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords)
                    .apply(self.stemming))
        return clean_X
    
    def transform_no_stem(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords))
        return clean_X

# Read in Data and Create Train and Test Sets

In [3]:
tweets = pd.read_csv('https://github.com/anilkeshwani/StatLearnProj/raw/master/Iason/climate_change_tweets_sample-2020-05-16-17-57.csv')
tweets.head()

,username,user_handle,date,retweets,favorites,text,label
0,WWF Climate & Energy,climateWWF,2020-04-28,11,22,Economic recovery and national climate pledges...,0
1,WWF Climate & Energy,climateWWF,2020-04-22,6,16,"In this difficult time, it’s hard to connect w...",0
2,WWF Climate & Energy,climateWWF,2020-04-01,43,69,"The decision to postpone # COP26, is unavoidab...",0
3,WWF Climate & Energy,climateWWF,2020-03-30,24,30,Japan - the world’s fifth largest emitter of g...,0
4,WWF Climate & Energy,climateWWF,2020-03-30,22,40,How can countries include # NatureBasedSolutio...,0


# Text Cleaning 

In [4]:
ct = CleanText()
tweets["text"] = ct.fit_transform(tweets.text)
tweets.to_csv("clean_tweets.csv") # save once processed
tweets = pd.read_csv("clean_tweets.csv") # read in instead
tweets = tweets.loc[(~tweets.text.isnull()), :]

# Splitting 

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(tweets.text, tweets.label, 
                                                    test_size=0.2, random_state=17, 
                                                    shuffle=True) # explicit default

[print(dat.head(3), dat.shape, end="\n"*2) for dat in [X_train, X_test, Y_train, Y_test]];

3642     might progress climat area not enough global s...
12695    trump crackdown politic scienc nasa climat div...
8451     no one would believ human panick fals climat m...
Name: text, dtype: object (14406,)

8376     nazi root environment climat chang fraud bbcne...
6111     interest democrat candid compar mani aspect cl...
13983    ittrademark imposs see global warm signal minn...
Name: text, dtype: object (3602,)

3642     0
12695    1
8451     1
Name: label, dtype: int64 (14406,)

8376     1
6111     0
13983    1
Name: label, dtype: int64 (3602,)



In [6]:
print(f"Training label counts: \n{Y_train.value_counts()}", end="\n"*2)
print(f"Test label counts: \n{Y_test.value_counts()}")

Training label counts: 
1    8433
0    5973
Name: label, dtype: int64

Test label counts: 
1    2138
0    1464
Name: label, dtype: int64


# Word Vectorisations 
FBOW

In [7]:
FBOW_vectorizer = CountVectorizer(stop_words = 'english', 
                                  binary=False, # Creates Word Frequency Vector; 
                                                # # np.unique(FBOW_train.toarray())
                                  min_df = 10)
FBOW_vectorizer.fit(X_train)
X_train = FBOW_vectorizer.transform(X_train)
X_test = FBOW_vectorizer.transform(X_test)

# Scaler

In [8]:
#

# CV

In [9]:
kfcv = KFold(n_splits=5,shuffle=True,random_state=101)

# Classifier

In [10]:
param_grid = {
            'penalty': ['none'],
            'C': [.001, .01, .1, 1, 10, 100, 1000],
            'solver': ['newton-cg', 'lbfgs','sag', 'saga'],
            'multi_class': ['ovr'],
            'max_iter' : [1000]
            }
clf = LogisticRegression()

# Grid Search

In [11]:
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                           cv=kfcv,n_jobs = -1, verbose =2, scoring='accuracy')
grid_search.fit(X_train,Y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed: 12.0min finished
C:\Users\Eleftheria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\Eleftheria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=KFold(n_splits=5, random_state=101, shuffle=True),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [1000], 'multi_class': ['ovr'],
                         'penalty': ['none'],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
             scoring='accuracy', verbose=2)

In [12]:
#print("Contains scores for all parameter combinations in param_grid. Each entry corresponds to one parameter setting. Each named tuple has the attributes: \n--> parameters : a dict of parameter settings \n--> mean_validation_score : the mean score over the cross-validation folds \n--> cv_validation_scores : the list of scores for each fold")
#print()
#print(grid_search.cv_results_)
#print()
print("Estimator that was chosen by the search, i.e. estimator which gave highest score (or smallest loss if specified) on the left out data")
print()
print(grid_search.best_estimator_)
print()
print("Score of best_estimator on the left out data.")
print()
print(round(grid_search.best_score_,3) )
print()
#print("Parameter setting that gave the best results on the hold out data.")
print()
#print(grid_search.best_estimator_.get_params)
print()
print("Scorer function used on the held out data to choose the best parameters for the model.")
print()
print(grid_search.scorer_)

Estimator that was chosen by the search, i.e. estimator which gave highest score (or smallest loss if specified) on the left out data

LogisticRegression(C=0.001, max_iter=1000, multi_class='ovr', penalty='none',
                   solver='saga')

Score of best_estimator on the left out data.

0.89



Scorer function used on the held out data to choose the best parameters for the model.

make_scorer(accuracy_score)


In [13]:
print('Training Mean Accuracy: {}'.format(round(grid_search.best_estimator_.score(X_train, Y_train))))

print("Test Mean Accuracy : {:.3f}".format(grid_search.best_estimator_.score(X_test, Y_test)))

Training Mean Accuracy: 1.0
Test Mean Accuracy : 0.896


In [14]:
proba_train = grid_search.predict_proba(X_train)[:, 1]
proba_test = grid_search.predict_proba(X_test)[:, 1]
BOW_logreg_noreg_brier_train = brier_score_loss(Y_train, proba_train)
BOW_logreg_noreg_brier_test = brier_score_loss(Y_test, proba_test)
print(f"Training (1 - Brier Score): {1 - round(BOW_logreg_noreg_brier_train, 3)}")
print(f"Test (1 - Brier Score): {1 - round(BOW_logreg_noreg_brier_test, 3)}")



Training (1 - Brier Score): 0.975
Test (1 - Brier Score): 0.919
